Import

In [1]:
import os
import openai
import sys

sys.path.append("../..")

from dotenv import load_dotenv, find_dotenv

env_path = find_dotenv()
print(f"Path of .env : {env_path}")


_ = load_dotenv(find_dotenv())  # read local .env file

openai.api_key = os.environ["OPENAI_API_KEY"]

Path of .env : /home/guillaume/Code/forestbot/.env


Document

In [5]:
from langchain_community.document_loaders import PyPDFLoader

# def load_pdf_with_metadata(file_path):
#     loader = PyPDFLoader(file_path)
#     documents = loader.load()
#     # Assign filename to each document's metadata
#     for document in documents:
#         document.metadata['filename'] = file_path
#     return documents

# docs1 = load_pdf_with_metadata("../pdf/EdAP 2020_EN.pdf")
# docs2 = load_pdf_with_metadata("../pdf/SOF book-web-rev3d-hires.pdf")

# documents = docs1 + docs2

loaders = [
    PyPDFLoader("../../pdf/EdAP 2020_EN.pdf"),
    PyPDFLoader("../../pdf/SOF book-web-rev3d-hires.pdf"),
]

documents = []
for loader in loaders:
    documents.extend(loader.load())

for document in documents:
    document.metadata["filename"] = document.metadata["source"]

# for document in documents:
#    document.metadata['filename'] = file_path

# Print metadata for checking
# for document in documents:
#    print(f"Document from: {document.metadata['filename']}")

In [6]:
# Print metadata for checking
# for document in documents:
#    print(f"Document from: {document.metadata['filename']}")
# print(documents[0])
# print(type(documents))
# print(documents[0].keys())
# print(dir(documents[0]))
# Afficher le contenu de la page du document
print("Contenu de la page :", documents[0].page_content)

# Afficher les métadonnées du document
print("Métadonnées :", documents[0].metadata)

Contenu de la page : STATE OF PROTECTED AREAS IN CENTRAL AFRICA 2020STATE OF 
PROTECTED
AREAS
IN CENTRAL 
AFRICA
2020

Métadonnées : {'source': '../../pdf/EdAP 2020_EN.pdf', 'page': 0, 'filename': '../../pdf/EdAP 2020_EN.pdf'}


Generate data set

In [8]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import OpenAIEmbeddings
from langchain_groq.chat_models import ChatGroq

import logging

# Configuration of logging
logging.basicConfig()
logger = logging.getLogger("tenacity")

# Define OpenAI models
# generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
# critic_llm = ChatOpenAI(model="gpt-4")

# Define Llama3/groq models
generator_llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")
critic_llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")


embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(generator_llm, critic_llm, embeddings)

In [11]:
testset = generator.generate_with_langchain_docs(
    documents[0:49],
    test_size=10,
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
)

embedding nodes:   0%|          | 0/114 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

Save dataset

In [14]:
df_testset = testset.to_pandas()
df_testset.to_csv(
    "evaluation_dataset_LLAMA3.csv", index=False
)  # `index=False` to not include label in the file

In [15]:
print(df_testset.columns)
print(df_testset.head)

Index(['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata',
       'episode_done'],
      dtype='object')
<bound method NDFrame.head of                                             question  \
0  Here is the question:\n\n"What role does endem...   
1  Here is a question that can be fully answered ...   
2  Here is the generated question:\n\n"What is th...   
3  Here is the question that can be fully answere...   
4  Here is a question that can be fully answered ...   
5  Here is a rewritten question that is more indi...   
6  Here is a rewritten version of the question:\n...   
7  Here is a rewritten version of the question th...   
8  Here is a rewritten question that is more indi...   
9  Here is the generated question:\n\n"What are t...   

                                            contexts  \
0  [11\nendemic poverty is contributing to wildli...   
1  [46\nFigure 12 – Donor funding for conservatio...   
2  [ allowed the \nDirectorate of Wildlife and Pr...   
3  [36